# SQL joining (merging) tables (exercises)

## Preparation

For this section you need `chinook.db` database file and working `%sql` magic.  
If you don't have it, please go back to the [previous section](../09_sql/connect_to_database.ipynb) and follow the instructions.  
The following code should not produce any errors:

In [ ]:
%load_ext sql
%sql sqlite:///chinook.db

## Exercise: Customer invoices

### Task

Use `LEFT JOIN` of tables `customers` and `invoice` to build a table listing all invoices per a customer.  
The table should have these columns:
- `CustomerId`
- `FirstName`
- `LastName`
- `InvoiceId`

What would be the primary key for this table?

### Solution

In [ ]:
%%sql
SELECT CustomerId, FirstName, LastName, InvoiceId FROM customers
  LEFT JOIN invoices USING (CustomerId)
  LIMIT 10

## Exercise: Customers without invoices?

### Task

Would customers without any invoice be visible in the result table produced above?

Check whether such customers exist there.

### Solution

`LEFT JOIN` with `customers` on the left side produces all customers in the output.  
Customers without any invoice would have `NULL` value in the `InvoiceId` field.

In [ ]:
%%sql
SELECT CustomerId, FirstName, LastName, InvoiceId FROM customers
  LEFT JOIN invoices USING (CustomerId)
  WHERE InvoiceId IS NULL 

## Exercise: Double join

### Task

*In short:* For each customer we want to know totals: how many tracks were bought and for what total price.

Build a temporary table `customer_summary` with the following columns:
The table should have these columns:
- `CustomerId` based on `customers`.
- `TotalTracksNum` - total number of tracks bought by the customer.
- `TotalPrice` - total sum of prices payed by the customer.

Use tables `customers`, `invoice` and `invoice_items`.

### Solution

Let's first take a solution of the first exercise - the table contains all customers and invoices corresponding to them.  
We want to extend this table with columns containing the totals.

In [ ]:
%%sql
SELECT CustomerId, InvoiceId FROM customers
  LEFT JOIN invoices USING (CustomerId)
  LIMIT 5

Let's generate totals per invoice based on `invoice_items` .  
This table will have a single row for each invoice:

In [ ]:
%%sql
SELECT 
    InvoiceId, 
    SUM(Quantity) AS InvoiceTracksNum, 
    SUM(UnitPrice) AS InvoiceTotalPrice 
  FROM invoice_items
  GROUP BY InvoiceId
  LIMIT 5

Now, let's apply `LEFT JOIN` to the above two tables (adjusting properly the columns to be selected). Moreover, let's store the result in a temporary table `invoice_summary`:

In [ ]:
%%sql
DROP TABLE IF EXISTS invoice_summary;
CREATE TEMP TABLE invoice_summary AS
    SELECT CustomerId, InvoiceId, InvoiceTracksNum, InvoiceTotalPrice FROM customers
      LEFT JOIN invoices USING (CustomerId)
      LEFT JOIN (
        SELECT 
            InvoiceId, 
            SUM(Quantity) AS InvoiceTracksNum, 
            SUM(UnitPrice) AS InvoiceTotalPrice 
          FROM invoice_items
          GROUP BY InvoiceId
      ) USING (InvoiceId);

SELECT * FROM invoice_summary LIMIT 5

Finally, let's aggregate per customer the data and store the result in the final temporary table:

In [ ]:
%%sql
DROP TABLE IF EXISTS customer_summary;
CREATE TEMP TABLE customer_summary AS
    SELECT 
        CustomerId,
        SUM(InvoiceTotalPrice) AS TotalPrice,
        SUM(InvoiceTracksNum) AS TotalTracksNum
      FROM invoice_summary
      GROUP BY CustomerId;

SELECT * FROM customer_summary LIMIT 5